In [1]:
import string
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
import pycrfsuite
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import spacy
import time

#Import PyTorch Framework
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
ct = CRFTagger()
nlp = spacy.load('en')

In [3]:
# Read in the train set data
#train_set  = pd.read_csv('Train_and_test_data/train.txt',sep=' ',names=['word','Brill','tag']).drop('Brill',1)
# Format train set data to tuples of a list of lists
#train_set = [[tuple(x) for x in train_set.values]]

In [4]:
#Training Code, Keep it commented unless retraining
#ct.train(train_set,'model.crf.tagger') 

#sentence = "Once upon a time there was a little girl called Cinderella. Cinderella is met a fairy who grants her a wish to be a Princess. Fairy said to Cinderella that she will become a Princess for one night"
#sentence = "Once upon a time there was a little girl called Alice . One day Alice met a Rabbit . The Rabbit said to Alice , follow me Alice I will carry you to my Den"

# Read the Test data from txt
sentence = ' '.join(open('Text_Passages2.txt','r').readlines()).rstrip("\n")
sentence

'Once on a time there was a Little Old Woman who lived in a Shoe. This shoe stood near a great forest, and was so large that it served as a house for the Old Lady and all her children, of which she had so many that she did not know what to do with them.\n \n But the Little Old Woman was very fond of her children, and they only thought of the best way to please her. Strong-arm, the eldest, cut down trees for firewood. Peter made baskets of wicker-work. Mark was chief gardener. Lizzie milked the cow, and Jenny taught the younger children to read.\n \n Now this Little Old Woman had not always lived in a Shoe. She and her family had once dwelt in a nice house covered with ivy, and her husband was a wood-cutter, like Strong-arm. But there lived in a huge castle beyond the forest, a fierce giant, who one day came and laid their house in ruins with his club; after which he carried off the poor wood-cutter to his castle beyond the forest. When the Little Old Woman came home, her house was in r

In [5]:
def process_text(passage):
    """
    1.Remove Punctuations and Special characters appear in book chapters
    2.Remove Stopwords
    3.Return Clean list of words
    """
    exclude_set = set(['“','”',':'])

    no_punctuation = [char for char in passage if char not in string.punctuation + "“”."]
    no_punctuation = ''.join(no_punctuation)
    no_punctuation = [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
    
    return no_punctuation

In [6]:
#remove stop words and punctuation
sentence = process_text(sentence)
sentence = ' '.join(sentence)
print(sentence)

time Little Old Woman lived Shoe shoe stood near great forest large served house Old Lady children many know Little Old Woman fond children thought best way please Strongarm eldest cut trees firewood Peter made baskets wickerwork Mark chief gardener Lizzie milked cow Jenny taught younger children read Little Old Woman always lived Shoe family dwelt nice house covered ivy husband woodcutter like Strongarm lived huge castle beyond forest fierce giant one day came laid house ruins club carried poor woodcutter castle beyond forest Little Old Woman came home house ruins husband seen


In [7]:
sentences = sent_tokenize(sentence)
word_list = [[]]
word_list.clear()
wlist = []
i = 0
j = 0

for sent in sentences:
    for word in sent.split():
        wlist.append(word)
        j += 1
        
    wlist.copy
    word_list.append(wlist.copy())
    wlist.clear()
    i += 1

In [8]:
#Set the model of previously trained data set
ct.set_model_file(model_file='model.crf.tagger')
word_list = ct.tag_sents(word_list)

#test_set = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag'], index_col=False).drop(['Brill','tag'],1)
#test_set = test_set.values.tolist()
#ct.tag_sents(test_set)


In [9]:
#test_setEval = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag']).drop('Brill',1)
#test_setEval = [[tuple(x) for x in test_setEval.values]]
#test the accuracy of the POS
#ct.evaluate(test_setEval)

In [10]:
#Convert list of tuples to a numpy array
word_list = np.array(word_list)
word_list = np.reshape(word_list, (-1,2))
word_list

array([['time', 'B-NP'],
       ['Little', 'I-NP'],
       ['Old', 'I-NP'],
       ['Woman', 'I-NP'],
       ['lived', 'B-VP'],
       ['Shoe', 'B-NP'],
       ['shoe', 'I-NP'],
       ['stood', 'I-NP'],
       ['near', 'B-PP'],
       ['great', 'B-NP'],
       ['forest', 'I-NP'],
       ['large', 'I-NP'],
       ['served', 'B-VP'],
       ['house', 'I-VP'],
       ['Old', 'B-NP'],
       ['Lady', 'I-NP'],
       ['children', 'I-NP'],
       ['many', 'I-NP'],
       ['know', 'B-VP'],
       ['Little', 'B-NP'],
       ['Old', 'I-NP'],
       ['Woman', 'I-NP'],
       ['fond', 'I-NP'],
       ['children', 'I-NP'],
       ['thought', 'I-NP'],
       ['best', 'I-NP'],
       ['way', 'I-NP'],
       ['please', 'I-NP'],
       ['Strongarm', 'I-NP'],
       ['eldest', 'I-NP'],
       ['cut', 'I-NP'],
       ['trees', 'I-NP'],
       ['firewood', 'I-NP'],
       ['Peter', 'I-NP'],
       ['made', 'B-VP'],
       ['baskets', 'B-NP'],
       ['wickerwork', 'I-NP'],
       ['Mark', 'I-NP'],
     

In [11]:
# Find S -> NP VP NP
# Returns if a subject is found
# pattern variable = [pat0, pat1, pat2]
def findSubject(pattern):
    #print('{} -> {} {}'.format(pattern[0], pattern[1], pattern[2]))
    if pattern[0] == 'NP' and pattern[1] == 'VP' and pattern[2] == 'NP':
        #if true pattern[0] is the subject of a sentence
        return True
    else:
        return False

In [12]:
#Extract the Subject -> NP VP NP to find the subjects in a sentence
#using Markov Chain Model

pattern = []
subjects = []
word_list_length = len(word_list)

for i in range(0, len(word_list)):
    #print(word_list[i][1].split('-')[1])
    try:
        if((word_list_length - i == 2)):
            break
        else:
            pattern = [word_list[i][1].split('-')[1], word_list[i+1][1].split('-')[1], word_list[i+2][1].split('-')[1]]
            # Call the function findSubject to identify potential subject elements and save them in an array
            if findSubject(pattern):
                # If returns true consider 1st element as a potential Subject
                print('Potential Subject Found at index {}'.format(i))
                print('Subject -> {} => {} {}'.format(word_list[i][0], word_list[i+1][0], word_list[i+2][0]))
                # Put the phrases in to a sentece and append it to an array.
                sub = [word_list[i][0],word_list[i+1][0],word_list[i+2][0]]
                subjects.append(' '.join(sub))
                print('--------------------')
            
    except IndexError:
        # Break the loop if IndexError occurs
        # Fail safe
        print("Out of Index")
        break;

Potential Subject Found at index 3
Subject -> Woman => lived Shoe
--------------------
Potential Subject Found at index 17
Subject -> many => know Little
--------------------
Potential Subject Found at index 33
Subject -> Peter => made baskets
--------------------
Potential Subject Found at index 57
Subject -> house => covered ivy
--------------------
Potential Subject Found at index 63
Subject -> Strongarm => lived huge
--------------------
Potential Subject Found at index 86
Subject -> Woman => came home
--------------------


In [13]:
# Find PERSONs in the filtered sentence using spaCy NER and build a DataFrame Object based on the data
# DataFrame structure -> Word | POS_TAG | Person

#Convert the subjects array to a 2D numpy array
subjects = np.array(subjects)
subjects = np.reshape(subjects, (-1,1))

refferingDataFrame = pd.DataFrame(columns=('Word_POS-TAG_Person','nullColumn'))

for i in range(0,len(subjects)):
    for j in range(0,len(subjects[i][0].split(" "))):
        wordBag = subjects[i][0].split(" ")
        # Find POS_TAG 
        pos = ct.tag([wordBag[j]])
        
        # Find IF word is Person
        person = [ent.label_ for ent in nlp(pos[0][0]).ents]
        person = ' '.join(person)
        if not person:
            person = ""
            
        #Add a row to the DataFrame with the retrived data
        refferingDataFrame.loc[len(refferingDataFrame)] = [(pos[0][0] +"_"+ pos[0][1].split('-')[1] +"_"+ person).rstrip('_'), ""]
        
refferingDataFrame

,Word_POS-TAG_Person,nullColumn
0,Woman_NP,
1,lived_VP,
2,Shoe_NP,
3,many_NP,
4,know_VP,
5,Little_NP,
6,Peter_SBAR_PERSON,
7,made_VP,
8,baskets_VP,
9,house_SBAR,


# Create the LSTM RNN

In [16]:
# The RNN is created using PyTorch Framework with Cuda disabled. The RNN will clasify the Question according the the catogiry
# To enable cuda add .cuda() method to torch.randn() method
lstm = nn.LSTM(3, 3)
inputs = [autograd.Variable(torch.randn(1, 3)) for _ in range(5)]
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))

for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))
out, hidden = lstm(inputs, hidden)
#print(out)
#print(hidden)

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

# Read training data from the csv
# Training data format => [(["Phrase"], ["Question"])]
unprocessed_data = pd.read_csv('Train_and_test_data/LSTM_train_set.csv',header=None)

# Transform the data in to a processable format
t_data_for_lstm = []
training_data = []
for phrase in unprocessed_data.itertuples():
    t_data_for_lstm.append(list(zip([[phrase[1]]], [[phrase[2]]])))

for i in range(0, len(t_data_for_lstm)):
    training_data.append(t_data_for_lstm[i][0])
    
word_to_ix = {}
tag_to_ix = {}

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for word1 in tags:
        if word1 not in tag_to_ix:
            tag_to_ix[word1] = len(tag_to_ix)
                            
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Create the model
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden  = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)), autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores
    
# set the variables to Train the model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

#Values Before Training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
t0 = time.time()
iterations = 300
progress_after = iterations/4

# Train the RNN for 300 iterations
for epoch in range(0,iterations):
    for sentence, tags in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        # Calculate the % of completion
        if(progress_after == epoch):
            print("Training {}% Completed".format((progress_after / iterations) * 100))
            progress_after = progress_after + (iterations / 4)
            
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
print("Training Complete!\nTotal Training time :", round(time.time()-t0, 2), "s\n")

# #Values after training        
# inputs = prepare_sequence(training_data[0][0], word_to_ix)
# tag_scores = model(inputs)

# print("Values After training")
# print(tag_scores)

Training 25.0% Completed
Training 50.0% Completed
Training 75.0% Completed
Training Complete!
Total Training time : 23.8 s



In [17]:
# Format the result generated by the LSTM by adding the relelevent Noun Phrases
def generate_questions(model, ref_data_frame):
    question = ""
    sentence = ""
    index = 0
    # for all the entries in the ref_data_frame predict a question
    for index, word_phrase in ref_data_frame.iterrows():
        
        # Predict the potentian sentence structure that can be used to generate the question.
        try:
            inputs = prepare_sequence([word_phrase[0]], word_to_ix)
            tag_scores = model(inputs)
        except KeyError:
            # ignore
            pass

        # Take the maximum probalilty, and based on the probabilty find index of value of the dictonary
        maxVal = max(tag_scores.data.numpy()[0])
        index_loc = 0
        probability_tag_scores = tag_scores.data.numpy().ravel()

        for i in range(0, len(probability_tag_scores)):
            if (probability_tag_scores.ravel()[i] == maxVal):
                index_loc = i

        # Travers the dictonary and identify the key value based on the probability predicted
        for key, value in tag_to_ix.items():
            if (value == index_loc):
                # format the Key to generate a meaningfull question
                # extract subject and object of the tested sentence
                sentence = word_phrase[0].split('_')
                # Check whether the question needs modification
                if "NP" in sentence:
                    question = key.replace("NP", sentence[0])
                    print(question)
                elif "VP" in sentence:
                    if "NP" in key and "N1" in key:
                        # Find the complete sentence matching for the verb
                        # Find the index in subjects which matches for verb
                        index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                        ref_subject = subjects.ravel()[index][0].split()
                        # Replace for words NP and NP1
                        question = key.replace("NP", ref_subject[0]) 
                        question = question.replace("N1",ref_subject[-1])
                        print(question)
                        
                    else:
                        # If "VP" but only one "NP"
                        index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                        ref_subject = subjects.ravel()[index][0].split()
                        question = key.replace("NP", ref_subject[0])
                        print(question)

In [18]:
generate_questions(model, refferingDataFrame)

Who is the Woman ?
Where did the Woman live ?
Where did the Shoe live ?
Where did the many live ?
Where did the many live ?
Who is the Little ?
What did Peter make ?
Who is Peter ?
What was the house ?
What was the Strongarm ?
Where did the Woman live ?
What was huge ?
Who is the Woman ?
Where did the Woman live ?
